In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/binary-classification-with-a-bank-dataset-clone/sample_submission.csv
/kaggle/input/binary-classification-with-a-bank-dataset-clone/train.csv
/kaggle/input/binary-classification-with-a-bank-dataset-clone/test.csv


In [45]:
train=pd.read_csv('/kaggle/input/binary-classification-with-a-bank-dataset-clone/train.csv')
test=pd.read_csv('/kaggle/input/binary-classification-with-a-bank-dataset-clone/test.csv')

In [46]:
train.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1


In [47]:
test.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,750000,32,blue-collar,married,secondary,no,1397,yes,no,unknown,21,may,224,1,-1,0,unknown
1,750001,44,management,married,tertiary,no,23,yes,no,cellular,3,apr,586,2,-1,0,unknown
2,750002,36,self-employed,married,primary,no,46,yes,yes,cellular,13,may,111,2,-1,0,unknown
3,750003,58,blue-collar,married,secondary,no,-1380,yes,yes,unknown,29,may,125,1,-1,0,unknown
4,750004,28,technician,single,secondary,no,1950,yes,no,cellular,22,jul,181,1,-1,0,unknown


In [48]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 18 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         750000 non-null  int64 
 1   age        750000 non-null  int64 
 2   job        750000 non-null  object
 3   marital    750000 non-null  object
 4   education  750000 non-null  object
 5   default    750000 non-null  object
 6   balance    750000 non-null  int64 
 7   housing    750000 non-null  object
 8   loan       750000 non-null  object
 9   contact    750000 non-null  object
 10  day        750000 non-null  int64 
 11  month      750000 non-null  object
 12  duration   750000 non-null  int64 
 13  campaign   750000 non-null  int64 
 14  pdays      750000 non-null  int64 
 15  previous   750000 non-null  int64 
 16  poutcome   750000 non-null  object
 17  y          750000 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 103.0+ MB


In [49]:
train.describe()

,id,age,balance,day,duration,campaign,pdays,previous,y
count,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000
mean,374999.500000,40.926395,1204.067397,16.117209,256.229144,2.577008,22.412733,0.298545,0.120651
std,216506.495284,10.098829,2836.096759,8.250832,272.555662,2.718514,77.319998,1.335926,0.325721
min,0.000000,18.000000,-8019.000000,1.000000,1.000000,1.000000,-1.000000,0.000000,0.000000
25%,187499.750000,33.000000,0.000000,9.000000,91.000000,1.000000,-1.000000,0.000000,0.000000
50%,374999.500000,39.000000,634.000000,17.000000,133.000000,2.000000,-1.000000,0.000000,0.000000
75%,562499.250000,48.000000,1390.000000,21.000000,361.000000,3.000000,-1.000000,0.000000,0.000000
max,749999.000000,95.000000,99717.000000,31.000000,4918.000000,63.000000,871.000000,200.000000,1.000000


In [50]:
train.isnull().sum()

id           0
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [51]:
from sklearn.preprocessing import LabelEncoder
cat_cols = ['job', 'marital', 'education', 'default', 'housing', 
            'loan', 'contact', 'month', 'poutcome']

for col in cat_cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col].astype(str))
    test[col] = le.transform(test[col].astype(str))

In [52]:
# splitting for validation
from sklearn.model_selection import  train_test_split
X = train.drop(['id', 'y'], axis=1)
y = train['y']
X_test = test.drop(['id'], axis=1)
test_ids = test['id']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

In [53]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)
X_full_scaled = scaler.fit_transform(X)

In [54]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, classification_report
# calculate class weight
class_weight = {0: 1.0, 1: (y_train==0).sum()/(y_train==1).sum()}


lr_model = LogisticRegression(
    max_iter=1000,
    C=0.1,  # regularization strength 
    class_weight='balanced',
    solver='lbfgs',
    random_state=42,
    n_jobs=-1
)

lr_model.fit(X_train_scaled, y_train)

LogisticRegression(C=0.1, class_weight='balanced', max_iter=1000, n_jobs=-1,
                   random_state=42)

In [55]:
lr_val_pred = lr_model.predict_proba(X_val_scaled)[:, 1]
lr_auc = roc_auc_score(y_val, lr_val_pred)

print(f"LR AUC: {lr_auc:.6f}")

LR AUC: 0.919672


In [56]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_estimators=500,max_depth=6,learning_rate=0.05,subsample=0.8,colsample_bytree=0.8,min_child_weight=3,gamma=0.1,eval_metric='auc',early_stopping_rounds=50,n_jobs=-1)

xgb_model.fit(X_train, y_train,eval_set=[(X_val, y_val)],verbose=False)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=50,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              feature_weights=None, gamma=0.1, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=3, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=-1, num_parallel_tree=None, ...)

In [57]:
from sklearn.metrics import roc_auc_score, roc_curve
xgb_val_pred = xgb_model.predict_proba(X_val)[:, 1]
xgb_auc = roc_auc_score(y_val, xgb_val_pred)
print(f"XGBOOST AUC SCORE: {xgb_auc:}")

XGBOOST AUC SCORE: 0.965610872290232


In [58]:
import lightgbm as lgb
from lightgbm import LGBMClassifier

lgb_model = LGBMClassifier(n_estimators=500,max_depth=6,learning_rate=0.05,num_leaves=50,subsample=0.8,colsample_bytree=0.8,min_child_samples=20,is_unbalance=True,n_jobs=-1,verbose=-1)

lgb_model.fit(X_train, y_train,eval_set=[(X_val, y_val)])

LGBMClassifier(colsample_bytree=0.8, is_unbalance=True, learning_rate=0.05,
               max_depth=6, n_estimators=500, n_jobs=-1, num_leaves=50,
               subsample=0.8, verbose=-1)

In [59]:
lgb_val_pred = lgb_model.predict_proba(X_val)[:, 1]
lgb_auc = roc_auc_score(y_val, lgb_val_pred)
print(f"LGBM AUC: {lgb_auc:}")

LGBM AUC: 0.965727204132785


In [60]:
from sklearn.ensemble import GradientBoostingClassifier

gb_model = GradientBoostingClassifier(n_estimators=200,learning_rate=0.1,max_depth=5,min_samples_split=20,min_samples_leaf=10,subsample=0.8,max_features='sqrt',verbose=0)
gb_model.fit(X_train, y_train)

GradientBoostingClassifier(max_depth=5, max_features='sqrt',
                           min_samples_leaf=10, min_samples_split=20,
                           n_estimators=200, subsample=0.8)

In [61]:
gb_val_pred = gb_model.predict_proba(X_val)[:, 1]
gb_auc = roc_auc_score(y_val, gb_val_pred)

print(f"GB AUC: {gb_auc}")

GB AUC: 0.9617950323979552


PICKING LGBM MODEL AS IT HAS HIGHEST SCORE

In [62]:
import lightgbm as lgb
from lightgbm import LGBMClassifier

final_model = LGBMClassifier(n_estimators=500,max_depth=6,learning_rate=0.05,num_leaves=50,subsample=0.8,colsample_bytree=0.8,min_child_samples=20,is_unbalance=True,n_jobs=-1,verbose=-1)

final_model.fit(X_train, y_train)

LGBMClassifier(colsample_bytree=0.8, is_unbalance=True, learning_rate=0.05,
               max_depth=6, n_estimators=500, n_jobs=-1, num_leaves=50,
               subsample=0.8, verbose=-1)

In [63]:

pred_test = final_model.predict(X_test)

In [64]:
# submission file
submission = pd.DataFrame({"id": test["id"],"y": pred_test})
submission.to_csv('submission.csv', index=False)


print("submitted successfully")

submitted successfully
